In [1]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: udacity-azure
Azure region: uaenorth
Subscription id: ac113154-2e6f-499c-9be7-09bd5fba4d3a
Resource group: mustafa.mohammed-rg


In [2]:
from azureml.core.compute import ComputeTarget, AmlCompute

cluster_name = "test-compute-cluster"

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###
from azureml.core.compute_target import ComputeTargetException
try:
    comp_trget = ComputeTarget(workspace=ws,name=cluster_name)
    print('already there')
except ComputeTargetException:  
    # Standard_E4s_v3
    # Standard_D2_V2
    comp_confg = AmlCompute.provisioning_configuration(vm_size='Standard_E4s_v3', max_nodes=4, min_nodes=1)
    comp_trget = ComputeTarget.create(ws, cluster_name, comp_confg)
    print('new one')

comp_trget.wait_for_completion(show_output=True)

new one
InProgress..
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded.......................
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [3]:
comp_trgets = ws.compute_targets
for comp_name,comp_type in comp_trgets.items():
    print(comp_name,comp_type.type,comp_type.provisioning_state)

notebook-compute ComputeInstance Succeeded
test-compute-cluster AmlCompute Succeeded


In [4]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
from azureml.core import Environment, ScriptRunConfig
import os

# Specify parameter sampler
# ps = ### YOUR CODE HERE ###
ps = RandomParameterSampling(
{
    '--C' : choice(0.03,0.3,3,10,30),
    '--max_iter' : choice(25,50,75,100)
})

# Specify a Policy
# policy = ### YOUR CODE HERE ###
policy = BanditPolicy(evaluation_interval = 3, slack_factor = 0.1)


if "training" not in os.listdir():
    os.mkdir("./training")

# Setup environment for your training run
sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='conda_dependencies.yml')

# Create a ScriptRunConfig Object to specify the configuration details of your training job
# src = ### YOUR CODE HERE ### 
# src = SKLearn(
#     source_directory = '.',
#     compute_target = comp_trget,
#     entry_script = 'train.py'
# )
#####
from azureml.core import Workspace, ScriptRunConfig, Environment

# curated_env_name = 'AzureML-PyTorch-1.6-GPU'
# pytorch_env = Environment.get(workspace=ws, name=curated_env_name)

arguments = [
    "--C",
    1.0,
    "--max_iter",
    100,
]

# compute_target = ws.compute_targets['my-cluster']
src = ScriptRunConfig(source_directory='.',
                      script='train.py',
                      arguments=arguments,
                      compute_target=comp_trget,
                      environment=sklearn_env)
#####

# Create a HyperDriveConfig using the src object, hyperparameter sampler, and policy.
# hyperdrive_config = ### YOUR CODE HERE ###
hyperdrive_config = HyperDriveConfig (
    hyperparameter_sampling=ps,
    primary_metric_name='Accuracy',
    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
    max_total_runs=40,
    max_concurrent_runs=5,
    max_duration_minutes=30,
    policy=policy,
    # estimator=src,
    run_config=src
)

In [5]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

from azureml.widgets import RunDetails
### YOUR CODE HERE ###
hyperdrive_submission = exp.submit(config=hyperdrive_config)
RunDetails(hyperdrive_submission).show()
hyperdrive_submission.wait_for_completion(show_output=True)

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_6f73b51b-9de2-46d7-a8d3-9a31844e86df
Web View: https://ml.azure.com/runs/HD_6f73b51b-9de2-46d7-a8d3-9a31844e86df?wsid=/subscriptions/ac113154-2e6f-499c-9be7-09bd5fba4d3a/resourcegroups/mustafa.mohammed-rg/workspaces/udacity-azure&tid=8f47ad71-44ca-48bf-afe3-56b9360a4495

Streaming azureml-logs/hyperdrive.txt

[2022-08-20T18:02:19.838035][GENERATOR][INFO]Trying to sample '5' jobs from the hyperparameter space
[2022-08-20T18:02:21.4341971Z][SCHEDULER][INFO]Scheduling job, id='HD_6f73b51b-9de2-46d7-a8d3-9a31844e86df_0' 
[2022-08-20T18:02:21.5356289Z][SCHEDULER][INFO]Scheduling job, id='HD_6f73b51b-9de2-46d7-a8d3-9a31844e86df_1' 
[2022-08-20T18:02:21.6733714Z][SCHEDULER][INFO]Scheduling job, id='HD_6f73b51b-9de2-46d7-a8d3-9a31844e86df_2' 
[2022-08-20T18:02:21.7948001Z][SCHEDULER][INFO]Scheduling job, id='HD_6f73b51b-9de2-46d7-a8d3-9a31844e86df_3' 
[2022-08-20T18:02:21.8899524Z][SCHEDULER][INFO]Scheduling job, id='HD_6f73b51b-9de2-46d7-a8d3-9a31844e86df_4' 
[2022-08-20T18:02:21.86

{'runId': 'HD_6f73b51b-9de2-46d7-a8d3-9a31844e86df',
 'target': 'test-compute-cluster',
 'status': 'Completed',
 'startTimeUtc': '2022-08-20T18:02:18.200335Z',
 'endTimeUtc': '2022-08-20T18:16:52.187083Z',
 'services': {},
 'properties': {'primary_metric_config': '{"name":"Accuracy","goal":"maximize"}',
  'resume_from': 'null',
  'runTemplate': 'HyperDrive',
  'azureml.runsource': 'hyperdrive',
  'platform': 'AML',
  'ContentSnapshotId': '573b0b1a-6478-42bb-a4f9-264d4c6e93e0',
  'user_agent': 'python/3.8.13 (Linux-5.15.0-1014-azure-x86_64-with-glibc2.17) msrest/0.6.21 Hyperdrive.Service/1.0.0 Hyperdrive.SDK/core.1.43.0',
  'space_size': '20',
  'score': '0.9060451565914057',
  'best_child_run_id': 'HD_6f73b51b-9de2-46d7-a8d3-9a31844e86df_1',
  'best_metric_status': 'Succeeded',
  'best_data_container_id': 'dcid.HD_6f73b51b-9de2-46d7-a8d3-9a31844e86df_1'},
 'inputDatasets': [],
 'outputDatasets': [],
 'runDefinition': {'configuration': None,
  'attribution': None,
  'telemetryValues': {

In [45]:
import joblib
# Get your best run and save the model from that run.

### YOUR CODE HERE ###
best_run = hyperdrive_submission.get_best_run_by_primary_metric()
print('ID : ',best_run.id)
print('Metrics : ',best_run.get_metrics())

final_model = best_run.register_model(model_name = 'hypermodel2',model_path = '/outputs/model.joblib')

ID :  HD_6f73b51b-9de2-46d7-a8d3-9a31844e86df_1
Metrics :  {'Regularization Strength:': 30.0, 'Max iterations:': 75, 'Accuracy': 0.9060451565914057}


In [41]:
import joblib
# Get your best run and save the model from that run.

### YOUR CODE HERE ###
best_run = hyperdrive_submission.get_best_run_by_primary_metric()
print(f'Best run ID: {best_run.id}')
print(f'Best metrics: \n{best_run.get_metrics()}')
print(best_run.get_details()['runDefinition']['arguments'])

file_name = 'outputs/hyperdrive_best_model8.joblib'

print(joblib.dump(value = best_run.id, filename=file_name))
print(run.upload_file(file_name, file_name))


best_model = best_run.register_model(model_name='bank-marketing-model', model_path=file_name)
# It gives me " ModelPathNotFoundException "

# print(best_run)

Best run ID: HD_6f73b51b-9de2-46d7-a8d3-9a31844e86df_1
Best metrics: 
{'Regularization Strength:': 30.0, 'Max iterations:': 75, 'Accuracy': 0.9060451565914057}
['--C', '1', '--max_iter', '100', '--C', '30', '--max_iter', '75']
['outputs/hyperdrive_best_model8.joblib']
{'additional_properties': {}, 'artifacts': {'outputs/hyperdrive_best_model8.joblib': <azureml._restclient.models.artifact_dto.ArtifactDto object at 0x7f6e81a2ebe0>}, 'artifact_content_information': {'outputs/hyperdrive_best_model8.joblib': <azureml._restclient.models.artifact_content_information_dto.ArtifactContentInformationDto object at 0x7f6e81a2e9d0>}, 'errors': {}}


ModelPathNotFoundException: ModelPathNotFoundException:
	Message: Could not locate the provided model_path outputs/hyperdrive_best_model8.joblib in the set of files uploaded to the run: ['logs/azureml/dataprep/0/backgroundProcess.log', 'logs/azureml/dataprep/0/backgroundProcess_Telemetry.log', 'logs/azureml/dataprep/0/rslex.log.2022-08-20-18', 'outputs/.amlignore', 'outputs/best_hyperdrive_model.joblib', 'outputs/model.joblib', 'system_logs/cs_capability/cs-capability.log', 'system_logs/hosttools_capability/hosttools-capability.log', 'system_logs/lifecycler/execution-wrapper.log', 'system_logs/lifecycler/lifecycler.log', 'system_logs/metrics_capability/metrics-capability.log', 'user_logs/std_log.txt']
                See https://aka.ms/run-logging for more details.
	InnerException None
	ErrorResponse 
{
    "error": {
        "message": "Could not locate the provided model_path outputs/hyperdrive_best_model8.joblib in the set of files uploaded to the run: ['logs/azureml/dataprep/0/back

In [22]:
# Get the best run and save the model from that run.

# get_best_run_by_primary_metric()
# Returns the best Run, or None if no child has the primary metric.
# best_run = hyperdrive_run.get_best_run_by_primary_metric()

# get_metrics()
# Returns the metrics from all the runs that were launched by this HyperDriveRun.
print("Best run metrics :",best_run.get_metrics())
print('===================================================')

# get_details()
# Returns a dictionary with the details for the run
print("Best run details :",best_run.get_details())
print('===================================================')

# get_file_names()
# Returns a list of the files that are stored in association with the run.

print("Best run file names :",best_run.get_file_names())
print('===================================================')

Best run metrics : {'Regularization Strength:': 30.0, 'Max iterations:': 75, 'Accuracy': 0.9060451565914057}
Best run details : {'runId': 'HD_6f73b51b-9de2-46d7-a8d3-9a31844e86df_1', 'target': 'test-compute-cluster', 'status': 'Completed', 'startTimeUtc': '2022-08-20T18:02:32.77739Z', 'endTimeUtc': '2022-08-20T18:03:45.558677Z', 'services': {}, 'properties': {'_azureml.ComputeTargetType': 'amlctrain', 'ContentSnapshotId': '573b0b1a-6478-42bb-a4f9-264d4c6e93e0', 'ProcessInfoFile': 'azureml-logs/process_info.json', 'ProcessStatusFile': 'azureml-logs/process_status.json'}, 'inputDatasets': [], 'outputDatasets': [], 'runDefinition': {'script': 'train.py', 'command': '', 'useAbsolutePath': False, 'arguments': ['--C', '1', '--max_iter', '100', '--C', '30', '--max_iter', '75'], 'sourceDirectoryDataStore': None, 'framework': 'Python', 'communicator': 'None', 'target': 'test-compute-cluster', 'dataReferences': {}, 'data': {}, 'outputData': {}, 'datacaches': [], 'jobName': None, 'maxRunDurationS

# AutoML

In [23]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"
### YOUR CODE HERE ###
ds = TabularDatasetFactory.from_delimited_files("https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv")

In [24]:
from train import clean_data
import pandas as pd
from sklearn.model_selection import train_test_split

# Use the clean_data function to clean your data.
x, y = clean_data(ds)### YOUR DATA OBJECT HERE ###
predict_col = pd.DataFrame(y)
full_data = pd.concat([x,predict_col],axis=1)
train_data, test_data = train_test_split(full_data,test_size=0.25)

In [25]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='classification',
    primary_metric='accuracy',
    training_data=train_data,
    label_column_name='y',
    n_cross_validations=7)


In [26]:
# Submit your automl run

### YOUR CODE HERE ###
run = exp.submit(config=automl_config, show_output=True)

No run_configuration provided, running on local with default configuration
Running in the active local environment.


Experiment,Id,Type,Status,Details Page,Docs Page
udacity-project,AutoML_26b2f25e-bbea-4c61-8b8e-0d02bed1b26f,automl,Preparing,Link to Azure Machine Learning studio,Link to Documentation


Current status: DatasetEvaluation. Gathering dataset statistics.
Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetFeaturizationCompleted. Completed fit featurizers and featurizing the dataset.
Current status: DatasetBalancing. Performing class balancing sweeping
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.

********************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely perceived positive effect of a model's accuracy because the input data has bias towards one cl

2022-08-20:20:04:58,78 INFO     [explanation_client.py:334] Using default datastore for uploads


In [30]:
# Retrieve and save your best automl model.

### YOUR CODE HERE ###
best_automl_run, best_model = run.get_output()
best_automl_run.register_model(model_name = "best_run_automl.pkl", model_path = './outputs/')
# best_run.download_file('outputs/model.pkl', 'hyperdrive_model.pkl')

Model(workspace=Workspace.create(name='udacity-azure', subscription_id='ac113154-2e6f-499c-9be7-09bd5fba4d3a', resource_group='mustafa.mohammed-rg'), name=best_run_automl.pkl, id=best_run_automl.pkl:4, version=4, tags={}, properties={})